### 마스크 착용여부 판단 모델(이진분류)


In [5]:
# 현재 작업 디렉토리 확인
!pwd
# 현재 작업 디렉토리의 폴더 내용 확인
!ls
# 폴더 이동
%cd ./drive/MyDrive/Colab\ Notebooks/22.04.11\ 머신러닝(교육실D)
# 폴더 생성 : mkdir
# 파일/폴더 복사 : cp
# 파일/폴더 이동 : mv
# 파일/폴더 삭제 : rm
# 추가 도구 설치 : apt-get

/content/drive
MyDrive
/content/drive/MyDrive/Colab Notebooks/22.04.11 머신러닝(교육실D)


In [7]:
# 작업 디렉토리 변경
%cd ./drive/MyDrive/Colab\ Notebooks/22.04.11\ 머신러닝(교육실D)

 data
'ex00_colab 사용하기.ipynb'
'ex01_비만도 데이터 학습하기.ipynb'
'ex02_주택가격 데이터 예측하기.ipynb'
'ex10_다중분류모델 만들기(3가지 동물 분류).ipynb'
'jupyter notebook'


In [ ]:
# 데이터셋 압축 풀기
!unzip ./data/dataset.zip -d ./data

#### 훈련데이터와 평가데이터로 나누기(사진파일형태 유지)

In [9]:
import os
import numpy as np

In [10]:
file_names_with_mask = os.listdir('./data/with_mask')
file_names_without_mask = os.listdir('./data/without_mask')

In [11]:
file_names_with_mask[:5]

['0-with-mask.jpg',
 '1-with-mask.jpg',
 '10-with-mask.jpg',
 '100-with-mask.jpg',
 '101-with-mask.jpg']

In [12]:
file_names_without_mask[:5]

['0.jpg', '1.jpg', '10.jpg', '100.jpg', '101.jpg']

In [29]:
# 난수 추출 기능
np.random.choice([10,5,99,100], size=2, replace=False) # replace=False -> 비복원 추출

array([10,  5])

In [33]:
np.random.seed(429)
test_with_mask = np.random.choice(file_names_with_mask,
                                  size=int(len(file_names_with_mask)*0.2),
                                  replace=False)
print(len(test_with_mask))
print(test_with_mask[:10])

138
['augmented_image_136.jpg' 'augmented_image_190.jpg'
 'augmented_image_139.jpg' '115-with-mask.jpg' 'augmented_image_77.jpg'
 'augmented_image_141.jpg' '314-with-mask.jpg' '476-with-mask.jpg'
 '200-with-mask.jpg' '243-with-mask.jpg']


In [34]:
np.random.seed(429)
test_without_mask = np.random.choice(file_names_without_mask,
                                  size=int(len(file_names_without_mask)*0.2),
                                  replace=False)

In [36]:
!mkdir ./data/train
!mkdir ./data/test
!cp -r ./data/with_mask ./data/train/with_mask
!cp -r ./data/without_mask ./data/train/without_mask

!mkdir ./data/test/with_mask
!mkdir ./data/test/without_mask

In [37]:
for name in test_with_mask :
  !mv ./data/train/with_mask/$name ./data/test/with_mask/

In [38]:
for name in test_without_mask :
  !mv ./data/train/without_mask/$name ./data/test/without_mask/

#### 모델링
- 전이학습 모델 : MobileNetV3
- 사진폴더부터 모델학습까지 한번에 이어지는 파이프라인을구축(tensorflow의 기능 활용)

In [50]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input # 사전학습모델용 전처리함수
from tensorflow.keras.preprocessing.image import ImageDataGenerator #이미지 증식

In [51]:
pre_trained_model = MobileNetV2(include_top=False,
                                     weights='imagenet',
                                     input_shape=(224,224,3))
pre_trained_model.trainable = False

9420800/9406464 [==============================] - 0s 0us/step


In [ ]:
pre_trained_model.summary()

In [52]:
mask_gen_train = ImageDataGenerator(rotation_range=20,
                                    zoom_range=0.15,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2,
                                    shear_range=0.15,
                                    horizontal_flip=True,
                                    preprocessing_function=preprocess_input)

In [53]:
mask_get_test = ImageDataGenerator(preprocessing_function=preprocess_input)

In [54]:
mask_model = Sequential() # 뼈대 생성
mask_model.add(pre_trained_model) # 사전학습모델(MobileNetV3)
mask_model.add(GlobalAveragePooling2D())
mask_model.add(Flatten()) # 추출된 특징을 1차원으로 만들어주는 레이어
mask_model.add(Dense(units=128,activation='relu')) # 중간층
mask_model.add(Dense(units=1,activation='sigmoid')) # 출력층

In [55]:
mask_model.compile(loss="binary_crossentropy",
                   optimizer='Adam',
                   metrics=['accuracy'])

In [56]:
mask_model.fit(mask_gen_train.flow_from_directory("./data/train",class_mode='binary'),
               epochs=3)

Found 1101 images belonging to 2 classes.
Epoch 1/3
35/35 [==============================] - 28s 675ms/step - loss: 0.0992 - accuracy: 0.9510
Epoch 2/3
35/35 [==============================] - 24s 690ms/step - loss: 0.0227 - accuracy: 0.9909
Epoch 3/3
35/35 [==============================] - 24s 690ms/step - loss: 0.0171 - accuracy: 0.9918


In [57]:
mask_model.evaluate(mask_get_test.flow_from_directory("./data/test", class_mode='binary'))

Found 275 images belonging to 2 classes.
9/9 [==============================] - 4s 307ms/step - loss: 0.0048 - accuracy: 1.0000


[0.004769233986735344, 1.0]

In [58]:
mask_model.save('./mask_detect_model.h5')